## Autores

| Nome | nUSP |
| :--- | :--- |
| Guilherme de Abreu Barreto | 12543033 |
| Lucas Eduardo Gulka Pulcinelli | 12547336 |
| Vinicio Yusuke Hayashibara | 13642797 |

In [ ]:
DATABASE = "postgres"
USER = "postgres"
PASSWORD = "postgres"
HOST = "postgres"
PORT = 5432
DATABASE_URI = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}/{DATABASE}"

In [ ]:
import pandas as pd
from enum import Enum
from math import sqrt
from import sqlalchemy import (
    BigInteger,
    UniqueConstraint as unique,
    PrimaryKeyConstraint as pkc,
    ForeignKey as fk,
    JSON,
    cast,
    create_engine,
    insert,
    text,
    func,
)
from sqlalchemy.orm import (
    Mapped,
    Session,
    declarative_base,
    relationship,
    sessiomaker,
    mapped_column as column,
)
from sqlalchemy.dialects.postgresql import composite, ENUM
from sqlalchemy.ext.hybrid import hybrid_method, hybrid_property
from typing import Optional

In [ ]:
def backref(back_populates: str) -> Mapped[Any]:
    return relationship(back_populates=back_populates)


def childOf(back_populates: str) -> Mapped[Any]:
    return relationship(
        back_populates=back_populates,
        cascade="all, delete-orphan",
    )

In [ ]:
class Households:
    def __init__(self, urban: int, rural: int) -> None:
        self.urban = urban
        self.rural = rural

    def __composite_values__(self) -> tuple[int, ...]:
        return (self.urban, self.rural)
        
    @hybrid_property
    def total(self):
        """Python-side property for total households."""
        return self.urban + self.rural

    @total.expression
    def total(cls):
        """SQL-side expression for querying total households."""
        return cls.urban + cls.rural

class Coordinate:
    """
    A geographic coordinate point with longitude (lon) and latitude (lat) components.
    """
    
    def __init__(self, longitude: float, latitude: float) -> None:
        self.lon = longitude
        self.lat = latitude

    @property
    def lon(self) -> float:
        return self._lon

    @lon.setter
    def lon(self, value: float) -> None:
        if not (-180 <= value <= 180):
            raise ValueError(f"Longitude must be between -180 and 180 degrees, got {value}")
        self._lon = value

    @property
    def lat(self) -> float:
        return self._lat

    @lat.setter
    def lat(self, value: float) -> None:
        if not (-90 <= value <= 90):
            raise ValueError(f"Latitude must be between -90 and 90 degrees, got {value}")
        self._lat = value

    def __composite_values__(self) -> tuple[float, ...]:
        return (self.lon, self.lat)

    def __eq__(self, other: "Coordinate") -> bool:
        return isinstance(other, Coordinate) and \
               other.lon = self.lon and other.lat == self.lat
    def __ne__ (self, other: "Coordinate") -> bool:
        return not self.__eq__(other)

    def _manhattan_distance(self, other: "Coordinate") -> float:
        return abs(self.lon - other.lon) + abs(self.lat - other.lat)

    def _euclidean_distance(self, other: "Coordinate") -> float:
        return sqrt(self.lon - other.lon)**2 + (self.lat - other.lat)**2)

    @hybrid_method
    def distance(self, other: "Coordinate", metric: str = 'euclidean') -> float
        """
        Calculate distance to another coordinate.
        
        Args:
            other: Coordinate instance
            metric: 'euclidean' or 'manhattan'
        
        Returns:
            Distance between coordinates
        """
        match metric:
            case 'euclidean':
                return self._euclidean_distance(other)
            case 'manhattan':
                return self._manhattan_distance(other)
            case _:
                raise ValueError("Metric must be 'euclidean' or 'manhattan'")

    @distance.expression
    def distance(
        cls,
        other_lon: float,
        other_lat: float,
        metric: str = 'euclidean'
    ):
        match metric:
            case 'euclidean':
                return func.sqrt(
                    (cls.x - other_x) * (cls.x - other_x) +
                    (cls.y - other_y) * (cls.y - other_y)
                )
            case 'manhattan':
                return func.abs(cls.x - other_x) + func.abs(cls.y - other_y)
            case _:
                raise ValueError("Metric must be 'euclidean' or 'manhattan'")


class Biomes:
    default: dict[str, float] = {
        biome: 0.0 for biome in [
            'amazon_rainforest',
            'atlantic_forest',
            'caatinga',
            'cerrado',
            'pantanal',
            'pampas'
        ]
    }

    def __init__(self, **kwargs) -> None:
        self.distribution = kargs

    def __composite_values__(self) -> Tuple[float, ...]:
        return tuple(getattr(self, biome) for biome in self.default.keys())

    @property
    def distribution(self) -> dict[str, float]:
        return {biome: getattr(self, biome) for biome in self.default.keys()}


    @distribution.setter
    def distribution(self, values: dict[str, float]) -> None:
        merged_values = {**self.default, **values}

        # Validation
        invalid_keys = set(merged_values.keys()) - set(self.default.keys())
        if invalid_keys:
            raise ValueError(f"Invalid biome types: {invalid_keys}. Valid types are: {list(self.default.keys())}")
        total = sum(merged_values.values())
        if 99.9 <= total <= 100.1:
            raise ValueError(f"Invalid biome distribution, totalling {total:.1f}%")
        self._distribution = merged_values

        for biome_type, value in merged_values.items():
            setattr(self, biome_type, value)

    @property
    def total(self) -> float:
        return sum(getattr(self, biome) for biome in self.default.keys())

In [ ]:
state_enum = ENUM(*[
    "AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", 
    "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", 
    "RS", "RO", "RR", "SC", "SP", "SE", "TO"
], name="state_enum")

region_enum = ENUM(*[
    "Norte", "Nordeste", "Centro-Oeste", "Sudeste", "Sul"    
], name="region_enum")

In [ ]:
Base = declarative_base()

class UTC_timezone(Base):
    __tablename__: "utc_timezones"

    # Attributes
    name: Mapped[str] = column(primary_key=True)
    offset: Mapped[int]

    # Relationships
    cities: Mapped[list["City"]] = backref('timezone')


class State(Base):
    __tablename__: "states"

    # Attributes
    id: Mapped[int] = column(primary_key=True)
    name: Mapped[str] = column(unique=True)
    uf: Mapped[state_enum] = column(unique=True)
    region: Mapped[region_enum]
    location: Mapped[Coordinate] = column(
        unique=True
        comment="The location of the state's centroid, in terms of longitude and latitude"
    )
    area: Mapped[float]
    biome_distribution: Mapped[Biomes]

    # Foreign keys
    capital_ibge_code: Mapped[int | None] = column(fk("cities.ibge_code"))

    # Relationships
    capital: Mapped[Optional["City"]] = relationship(
        foreign_keys="State.capital_ibge_code"        
    )
    cities: Mapped[list["City"]] = childOf("state")
    


class City(Base):
    __tablename__: "cities"

    # Attributes
    id: Mapped[int]
    name: Mapped[str]
    location: Mapped[Coordinate] = column(
        unique=True
        comment="The location of the state's centroid, in terms of longitude and latitude"
    )
    ddd: Mapped[int]
    households: Mapped[Households]
    population_race: Mapped[dict] = Column(JSON)
    population_education: Mapped[dict] = Column(JSON)
    

    # Foreign keys
    timezone_name: Mapped[str] = column(fk("utc_timezones.name"))
    state_id: Mapped[int] = column(fk("states.id"))

    # Relationships
    timezone: Mapped["UTC_Timezone"] = backref("cities")
    state: Mapped["State"] = backref("cities")

    @hybrid_property
    def ibge_code(self):
        """Python-side property to get the IBGE code."""
        return int(f"{self.state_id}{self.id}")

    @ibge_code.expression
    def ibge_code(cls):
        """SQL-side expression for querying."""
        return cast(
            func.concat(
                cast(cls.state_id, String),
                cast(cls.id, String)
            ),
            BigInteger
        )

    __table_args: tuple[pkc,] = (
        pkc("state_id", "id")
    )
    

